### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [29]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_wesad as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


phases = dr.Phases.PHASE_ORDER


In [30]:
importlib.reload(train)

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    # train.Metrics.MEAN_SCL, 
    # train.Metrics.SCR_RATE, 
# ]
] + train.Metrics.STATISTICAL

In [31]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


model_phases = dr.Phases.PHASE_ORDER

label_type = "stai"
threshold = "fixed"

x, y = train.Train_WESAD.get_wesad_data(
    metrics, model_phases, verbose=False, label_type=label_type, threshold=threshold, 
    normalize=True, combine_phases=False, standardize=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.92770573 0.93406991 0.93227043 0.93287973 0.93583125 0.92525212
 0.93409531 0.93554992 0.93543479 0.93296106 0.92886956 0.93336931
 0.93364943 0.93391004 0.93589931 0.93396605 0.93706911 0.93827282
 0.93369087 0.935103   0.93935913 0.93788858 0.93942398 0.9352294
 0.93400888        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [53 16], Predictions: [0 1], [58 11]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [53 16], Predictions: [0 1], [51 18]
Training RF ...
Model RF, Actual: [0 1], [53 16], Predictions: [0 1], [60  9]
Training XGB ...
Model XGB, Actual: [0 1], [53 16], Predictions: [0 1], [57 12]

None


[('ecg_mean', 98), ('ecg_kurtosis', 89), ('bpm', 87), ('ecg_iqr', 44), ('rmssd', 29), ('hf_rr', 28), ('eda_mean', 27), ('eda_var', 21), ('ecg_rms', 19), ('eda_kurtosis', 19), ('eda_std', 19), ('e

One or more of the test scores are non-finite: [0.92041878 0.91590123 0.91557153 0.91229185 0.90992113 0.91572838
 0.90841385 0.90879441 0.90902239 0.90581355 0.92138558 0.91146684
 0.90934837 0.90713177 0.90264772 0.91476192 0.90576878 0.90173607
 0.89670635 0.89640989 0.91932902 0.90993198 0.90432901 0.90148535
 0.90182947        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 10, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 16], Predictions: [0 1], [65  5]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 16], Predictions: [0 1], [52 18]
Training RF ...
Model RF, Actual: [0 1], [54 16], Predictions: [0 1], [65  5]
Training XGB ...
Model XGB, Actual: [0 1], [54 16], Predictions: [0 1], [56 14]

None


[('ecg_kurtosis', 166), ('bpm', 135), ('ecg_mean', 112), ('ecg_iqr', 65), ('hf_rr', 52), ('eda_skew', 37), ('ecg_median', 30), ('rmssd', 29), ('eda_mean', 28), ('ecg_var', 25), ('eda_var', 25), (

...

Grid search for XGB ...
SVM: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 20, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 7, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 16], Predictions: [0 1], [55 15]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 16], Predictions: [0 1], [55 15]
Training RF ...
Model RF, Actual: [0 1], [54 16], Predictions: [0 1], [55 15]
Training XGB ...
Model XGB, Actual: [0 1], [54 16], Predictions: [0 1], [55 15]

None


[('ecg_kurtosis', 327), ('bpm', 225), ('ecg_mean', 201), ('ecg_iqr', 121), ('eda_kurtosis', 87), ('ecg_skew', 84), ('eda_mean', 70), ('eda_skew', 62), ('rmssd', 61), ('ecg_median', 61), ('hf_rr', 56)

One or more of the test scores are non-finite: [0.85994265 0.90717259 0.89386483 0.88892865 0.88724129 0.84688357
 0.90095623 0.89242091 0.8942731  0.89304313 0.85342348 0.90362587
 0.8952248  0.89568953 0.89521823 0.87937978 0.91015654 0.9015252
 0.90392835 0.9000939  0.87852895 0.90380129 0.90411037 0.90158967
 0.90096864        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 20, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 16], Predictions: [0 1], [58 12]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 16], Predictions: [0 1], [51 19]
Training RF ...
Model RF, Actual: [0 1], [54 16], Predictions: [0 1], [47 23]
Training XGB ...
Model XGB, Actual: [0 1], [54 16], Predictions: [0 1], [50 20]

None


[('ecg_kurtosis', 241), ('bpm', 227), ('ecg_mean', 195), ('ecg_iqr', 133), ('ecg_skew', 97), ('eda_mean', 73), ('rmssd', 64), ('eda_kurtosis', 62), ('ecg_rms', 49), ('ecg_median', 45), ('eda_var'

One or more of the test scores are non-finite: [0.88491538 0.88341153 0.88770833 0.89967315 0.9002913  0.89993628
 0.89986883 0.89333351 0.89759571 0.89941453 0.90024024 0.90480146
 0.8988132  0.90349433 0.90218113 0.88850907 0.89625928 0.89475803
 0.90197678 0.9043659  0.90588639 0.88969127 0.88880445 0.9012767
 0.89751925        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 10, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [53 16], Predictions: [0 1], [56 13]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [53 16], Predictions: [0 1], [53 16]
Training RF ...
Model RF, Actual: [0 1], [53 16], Predictions: [0 1], [54 15]
Training XGB ...
Model XGB, Actual: [0 1], [53 16], Predictions: [0 1], [53 16]

None


[('ecg_kurtosis', 257), ('ecg_mean', 238), ('bpm', 226), ('ecg_iqr', 135), ('eda_skew', 83), ('ecg_skew', 81), ('ecg_median', 80), ('ecg_rms', 72), ('eda_kurtosis', 61), ('rmssd', 55), ('hf_rr', 51),

In [32]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

from sklearn.ensemble import VotingClassifier

test_size = 0.15
folds = 5

cv, x_train, y_train, x_test, y_test = train.kfold_train_test_split(x, y, test_size, is_resample=False, folds=folds)

estimators = [(name, best_models[name]) for name in best_models.keys()]
ensemble = VotingClassifier(estimators, voting="hard")
ensemble.fit(x_train, y_train.loc[:, "label"])
ensemble.score(x_test, y_test.loc[:, "label"])

0.8095238095238095